# Ray-tracer setup
---

In [1]:
model_name = 'model.txt'

Download ascii output of Jolien's cool Phantom model.

In [2]:
# ! wget https://owncloud.ster.kuleuven.be/index.php/s/6mCZjZ2erTsXq5Y/download -O $model_name

In [2]:
import numpy as np
import k3d

from scipy.spatial import Delaunay, cKDTree   # Finding neighbors

In [6]:
# Read only the coordinate of the points in the file
(x,y,z, density) = np.loadtxt(model_name, skiprows=14, usecols=(0,1,2,5), unpack=True)
# Put them in a nice numpy array
points = np.array((x,y,z)).transpose()
# Convert density to cgs units (see model file)
density *= 5.941031250291510e-07

In [5]:
# Create a Delaunay tetrahedralisation of the points
# (We use this to find the nearest neighbors)
# https://en.wikipedia.org/wiki/Delaunay_triangulation
delaunay = Delaunay(points)

In [6]:
# Extract Delaunay vertices (= Voronoi neighbors)
(indptr, indices) = delaunay.vertex_neighbor_vertices
neighbors = [list(indices[indptr[k]:indptr[k+1]]) for k in range(len(points))]
# neighbours is now a list of lists in which the i'th lists containts the list
# of indices of the nearest neighbours of the i'th particle.

In [7]:
# Visualise the points
plot = k3d.plot(name='points')
plt_points = k3d.points(positions=points.astype(np.float32))
plot += plt_points
plt_points.shader='3d'
plot.display()

Output()

# Attempt to make volume rendering of the model

In [10]:
from scipy.interpolate import griddata
#
ngridx = 100
ngridy = 100
ngridz = 100

# Create image grid values
xs = np.linspace(np.min(x), np.max(x), ngridx)
ys = np.linspace(np.min(y), np.max(y), ngridy)
zs = np.linspace(np.min(z), np.max(z), ngridz)

cube = griddata((x, y, z), density, (xs[None,None,:], ys[None,:,None], zs[:,None,None]), method='nearest')

In [20]:
plot = k3d.plot()


volume = k3d.volume(
    cube.astype(np.float32),
    # alpha_coef=1000,
    # shadow='off', #dynamic is a fancy option but require a good GPU
    # samples=600,
    # shadow_res=128,
    # shadow_delay=50,
    # color_range=[150, 750],
    # color_map=(np.array(k3d.colormaps.matplotlib_color_maps.Gist_heat).reshape(-1, 4)
    #            * np.array([1, 1.75, 1.75, 1.75])).astype(np.float32)
)

volume.transform.bounds = [np.min(x), np.max(x),
                           np.min(y), np.max(y),
                           np.min(z), np.max(z)]
plot.display()

Output()